In [9]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd

In [10]:
def read_images():
    training_images = []
    i = 0
    DIR = "E:\Galaxy Zoo\Data\images_training_rev1\images_training_rev1"
    for img in os.listdir(DIR):
        im = cv2.imread(os.path.join(DIR,img))
        im = im[106:106*3,106:106*3,:]          #cropping 424x424->212x212
        im = cv2.resize(im,(106,106), interpolation=cv2.INTER_CUBIC)   #downsampling to half resolution 106x106
        training_images.append(im)
        i+=1
        if i==10000:
            break
        
    return training_images
    

In [11]:
training_images = read_images()
training_images = np.array(training_images).reshape(-1,212,212,3)

In [12]:
X_train = training_images[:8000]
x_test = training_images[8000:]
df = pd.read_csv("training_solutions_rev1/training_solutions_rev1.csv")
y_train = df.drop('GalaxyID', axis = 1).values
y_train = y_train[:10000]
y_test = y_train[8000:10000]
y_train = y_train[:8000]

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D

In [14]:
X_train = tf.keras.utils.normalize(X_train)

In [15]:
X_train.shape

(2500, 212, 212, 3)

In [28]:
model = Sequential()

model.add(Conv2D((64), kernel_size = (3,3), input_shape = (212,212,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (3,3), strides = 3))

model.add(Conv2D((64), kernel_size = (3,3), input_shape = (212,212,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (3,3), strides = 3))

model.add(Conv2D((32), kernel_size = (3,3), input_shape = (212,212,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (3,3), strides = 3))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(37))
model.add(Activation("sigmoid"))

model.compile(optimizer = 'adam', loss = "sparse_categorical_crossentropy")


In [29]:
y_train = y_train[:2500]

In [30]:
model.fit(X_train,y_train,epochs = 50)

Epoch 1/50


InvalidArgumentError: logits and labels must have the same first dimension, got logits shape [32,37] and labels shape [1184]
	 [[{{node loss_2/activation_20_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits}}]]